In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from joblib import dump, load
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoModel, set_seed
from datasets import load_dataset
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from joblib import dump
import numpy as np
import evaluate

C:\Users\SEBASTIAN\PycharmProjects\MachinePaper\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
set_seed(42)
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load dataset

In [3]:
from datasets import load_dataset

# Load a custom CSV file
data_files = {"train": "../../data/train_data.csv", "valid":"../../data/val_data.csv","test": "../../data/test_data.csv"}
dataset = load_dataset("csv", data_files=data_files)

# Inspect the first few samples
print(dataset["train"][0])
# Example: Mapping string labels to integers
label_mapping = {
    'notcb': 0,
    'gender': 1,
    'ethnicity': 2,
    'religion': 3,
    'age' : 4,
    'other': 5
}
dataset = dataset.map(lambda x: {"label": label_mapping[x["label"]]})

# Verify the mapping
print(dataset)

{'text': '[USER]: FUCK OBAMA, dumb ass nigger hahahaha you mad [HASHTAG] [HASHTAG]', 'label': 'ethnicity'}
DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 32638
    })
    valid: Dataset({
        features: ['text', 'label'],
        num_rows: 8160
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 7200
    })
})


In [4]:
model_name = "roberta-base"

# tokenize

In [5]:
dataset["valid"]

Dataset({
    features: ['text', 'label'],
    num_rows: 8160
})

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples): #padding false becase of data collator later
    return tokenizer(examples["text"], padding=False, truncation=True, max_length=200)

# Apply the tokenizer to the dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Inspect tokenized samples
print(tokenized_datasets["train"][0])

Map: 100%|██████████| 7200/7200 [00:00<00:00, 35715.16 examples/s]

{'text': '[USER]: FUCK OBAMA, dumb ass nigger hahahaha you mad [HASHTAG] [HASHTAG]', 'label': 2, 'input_ids': [0, 10975, 47955, 42645, 46997, 28214, 19455, 6, 16881, 8446, 295, 41956, 2489, 298, 895, 11695, 47, 7758, 646, 725, 2336, 14469, 3450, 742, 646, 725, 2336, 14469, 3450, 742, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [7]:
metric = evaluate.load("recall")


def model_init():
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=6)
    print(model.config)
    return model

# How to Custom Model

In [8]:
# import torch.nn as nn
#
# class CustomBERTModel(nn.Module):
#     def __init__(self, pretrained_model_name, num_labels):
#         super(CustomBERTModel, self).__init__()
#         self.bert = AutoModel.from_pretrained(pretrained_model_name)
#         self.dropout = nn.Dropout(0.3)
#         self.fc = nn.Linear(self.bert.config.hidden_size, num_labels)
#
#     def forward(self, input_ids, attention_mask):
#         output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
#         pooled_output = self.dropout(output[1])  # Applying dropout
#         logits = self.fc(pooled_output)  # Adding a fully connected layer
#         return logits
#
# # Initialize the custom model
# custom_model = CustomBERTModel("bert-base-uncased", num_labels=4)

## hyperparameter tuning

In [13]:
import optuna
from optuna.storages import RDBStorage
import os

# Define persistent storage
storage = RDBStorage("sqlite:///optuna_trials.db")

study = optuna.create_study(
    study_name=f"{model_name}-opt-study", direction="maximize", storage=storage, load_if_exists=True
)

# set the wandb project where this run will be logged
os.environ["WANDB_PROJECT"]="cyberbullying-bert-based-finetuning"

# save your trained model checkpoint to wandb
os.environ["WANDB_LOG_MODEL"]="end"

# turn off watch to log faster
os.environ["WANDB_WATCH"]="false"


[I 2025-08-08 19:03:46,799] Using an existing study with name 'roberta-opt-study' instead of creating a new one.


In [10]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [14]:
import wandb
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import Trainer, TrainingArguments


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    # Calculate accuracy
    accuracy = accuracy_score(labels, predictions)

    # Calculate precision, recall, f1 with macro averaging (treats all classes equally)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, predictions, average='macro', zero_division=0
    )

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

def compute_objective(metrics):
    return metrics["eval_recall"]


wandb.init(project="cyberbullying-bert-based-finetuning", name=f"{model_name}-opt-study")

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    save_total_limit=3,
    logging_strategy="epoch",
    num_train_epochs=3,
    report_to="wandb",
    logging_dir="./logs",
    run_name=f"{model_name}-opt-study",
)


trainer = Trainer(
    model_init=model_init,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["valid"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.54.0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}



In [18]:
def optuna_hp_space(trial):
    return {
        "learning_rate" : trial.suggest_float("learning_rate", 1e-5, 3e-5, log=True),
        "per_device_train_batch_size" : trial.suggest_categorical("per_device_train_batch_size", [16, 32, 64]),
        "num_train_epochs" : trial.suggest_int("num_train_epochs", 5, 10),
        "warmup_steps" : trial.suggest_int("warmup_steps", 0, 500),
        "weight_decay" : trial.suggest_float("weight_decay", 0.01, 0.1)
}


best_run = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=optuna_hp_space,
    n_trials=10,
    compute_objective=compute_objective,
    study_name=f"{model_name}-opt-study",
    storage="sqlite:///optuna_trials.db",
    load_if_exists=True,
)

print(best_run)

[I 2025-08-08 19:19:25,397] Using an existing study with name 'roberta-opt-study' instead of creating a new one.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.54.0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}



C:\Users\SEBASTIAN\PycharmProjects\MachinePaper\.venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
C:\Users\SEBASTIAN\PycharmProjects\MachinePaper\.venv\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Epoch,Training Loss,Validation Loss


[W 2025-08-08 19:19:59,063] Trial 2 failed with parameters: {'learning_rate': 1.3657379780940003e-05, 'per_device_train_batch_size': 32, 'num_train_epochs': 8, 'warmup_steps': 201, 'weight_decay': 0.08236181106735314} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "C:\Users\SEBASTIAN\PycharmProjects\MachinePaper\.venv\Lib\site-packages\optuna\study\_optimize.py", line 201, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\SEBASTIAN\PycharmProjects\MachinePaper\.venv\Lib\site-packages\transformers\integrations\integration_utils.py", line 265, in _objective
    trainer.train(resume_from_checkpoint=checkpoint, trial=trial)
    ~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\SEBASTIAN\PycharmProjects\MachinePaper\.venv\Lib\site-packages\transformers\trainer.py", line 2237, in train
    return inner_training_loop(
        args=args,
    ...<2 lines>...
        ignore_keys_for_eval=ignore_keys_for_e

KeyboardInterrupt: 

In [ ]:
import optuna
from optuna.visualization.matplotlib import (
    plot_optimization_history,
    plot_intermediate_values,
    plot_param_importances,
)
import matplotlib.pyplot as plt

# Load the study from RDB storage
storage = optuna.storages.RDBStorage("sqlite:///optuna_trials.db")

study = optuna.load_study(study_name=f"{model_name}-opt-study", storage=storage)

# Plot optimization history
ax1 = plot_optimization_history(study)
plt.show()
ax1.figure.savefig("optimization_history.png")

# Plot intermediate values (if using pruning and intermediate reports)
ax2 = plot_intermediate_values(study)
plt.show()
ax2.figure.savefig("intermediate_values.png")

# Plot parameter importances
ax3 = plot_param_importances(study)
plt.show()
ax3.figure.savefig("param_importances.png")

# Training optimized

In [ ]:
# from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
#
# # Define the model
# model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=6)
#
# # Load best hyperparameters (already defined earlier as best_hparams)
# training_args = TrainingArguments(
#     output_dir="./final_model",
#     learning_rate=best_hparams["learning_rate"],
#     per_device_train_batch_size=best_hparams["per_device_train_batch_size"],
#     weight_decay=best_hparams["weight_decay"],
#     eval_strategy="epoch",
#     save_strategy="epoch",
#     load_best_model_at_end=True,
#     logging_strategy="epoch",
#     num_train_epochs=best_hparams["num_train_epochs"],
#     warmup_steps=best_hparams["warmup_steps"],
#     report_to="wandb",
#     run_name="final_run_with_best_hparams",
# )
#
# # Create Trainer
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=tokenized_datasets["train"],
#     eval_dataset=tokenized_datasets["valid"],
#     processing_class=tokenizer,
#     data_collator=data_collator,
#     compute_metrics=compute_metrics,
# )
# from transformers import EarlyStoppingCallback
#
# trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=2))
# # Train
# trainer.train()
#
# # Save the model
# trainer.save_model("./final_model")

In [ ]:
# from matplotlib import pyplot as plt
# from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
#
# # Generate predictions
# predictions = trainer.predict(tokenized_datasets["test"])
# predicted_labels = predictions.predictions.argmax(axis=-1)
#
# # Classification report
# print(classification_report(tokenized_datasets["test"]["label"], predicted_labels))
#
# # Confusion matrix
# cm = confusion_matrix(tokenized_datasets["test"]["label"], predicted_labels)
# disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["nocb", "gender","ethnicity", "religion", "age","other"])
# disp.plot(cmap="Blues")  # Optional: set a color map
# plt.tight_layout()
# plt.savefig("confusion_matrix.png", dpi=300)  # You can change the name or dpi as needed
# plt.close()  # Close the plot to free memory if you're in a loop

In [ ]:
# # Inspect misclassified samples
# for idx, (pred, label) in enumerate(zip(predicted_labels, tokenized_datasets["test"]["label"])):
#     if pred != label:
#         print(f"Index: {idx}, Predicted: {pred}, Actual: {label}")
#         print(tokenized_datasets["test"][idx]["text"])

In [ ]:
tokenizer.save_pretrained("./final_model")
print("Model saved successfully!")